In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import json
from tokenizer import Tokenizer 

torch.manual_seed(1337)

import json

# Load vocabulary from JSON file
with open('vocab.json', 'r') as json_file:
    vocab = json.load(json_file)

tokenizer = Tokenizer(vocab)

class SelfAttentionModel(nn.Module):

    def __init__ (self, embedding_size, embedding_dim):
        super().__init__()
        self.token_embedding_table = nn.Embedding(embedding_size, embedding_dim)

    def forward(self, input, target):
        logits = self.token_embedding_table(input) #128 x 94 -> num_mels x max_num_frames
        loss = torch.nn.functional.l1_loss(input, target, size_average=None, reduce=None, reduction='mean')
        return logits

